In [ ]:
import os
import gokart
from elasticsearch import Elasticsearch

from qrelllm.eval import ndcg_compare_report
from qrelllm.queries import LoadQueries
from qrelllm.llm.vertex import TestCollection
from qrelllm.format import format_test_collection
from qrelllm.es.es import ping, index, run_with_kuromoji, run_with_ngram

In [ ]:
project = os.getenv("GOOGLE_CLOUD_PROJECT_ID")
location = os.getenv("GOOGLE_CLOUD_LOCATION")

In [ ]:
index_name = "docs"
client = Elasticsearch(hosts=["http://localhost:9200"])
ping(client)

queries = LoadQueries(csv_file_path="../data/queries.csv")
testcollection = TestCollection(
    project=project,
    location=location,
    queries=queries,
    size=300
)
df = gokart.build(testcollection)
df = format_test_collection(df)

index(client, df)

run1_df = run_with_ngram(client, index_name, df)
run2_df = run_with_kuromoji(client, index_name, df)

ndcg_compare_report(df, run1_df, run2_df)